## Import các thư viện cần thiết

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import datetime

---

## Thu thập dữ liệu

Trong bài này, ta sử dụng thư viện `Beautiful Soup` để cào dữ liệu từ trang [Wolrdometer](https://www.worldometers.info/coronavirus/) vào mỗi **23:55** hằng ngày.

Chú ý rằng, ta được phép sử dụng bộ dữ liệu này cho mục đích học tập.

---

## Khám phá dữ liệu

### Đọc dữ liệu từ file csv vào dataframe

In [6]:
data = pd.read_csv('data/covid_data.csv')
data.head()

,Date,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Active Cases,Serious Critical,Tot Cases/1M pop,Deaths/1M pop,Total Tests,Tests/1M pop,Population
0,2022-04-19,USA,82377156,NaN,1015790.0,NaN,80244093.0,128.0,1117273.0,1438.0,246284.0,3037.0,996639125.0,2979667.0,3.344800e+08
1,2022-04-19,India,43045527,NaN,521996.0,NaN,42511701.0,NaN,11830.0,698.0,30652.0,372.0,832506755.0,592823.0,1.404310e+09
2,2022-04-19,Brazil,30261088,NaN,662076.0,NaN,29262483.0,NaN,336529.0,8318.0,140575.0,3076.0,63776166.0,296266.0,2.152663e+08
3,2022-04-19,France,27790834,NaN,144205.0,NaN,25132303.0,NaN,2514326.0,1541.0,424079.0,2201.0,260504402.0,3975208.0,6.553227e+07
4,2022-04-19,Germany,23502641,45045.0,133463.0,26.0,20347900.0,262100.0,3021278.0,1980.0,278917.0,1584.0,122332384.0,1451777.0,8.426390e+07


### Dữ liệu gồm bao nhiều dòng, bao nhiêu cột?

In [7]:
num_rows = data.shape[0]
num_cols = data.shape[1]
print('Số dòng: ', num_rows)
print('Số cột: ', num_cols)

Số dòng:  228
Số cột:  15


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng tương ứng với thông tin số ca nhiễm của một nước trong một ngày. 

Có vẻ như không xảy ra các dòng có ý nghĩa khác nhau

### Dữ liệu có các dòng bị lặp không?

In [8]:
duplicated_rows = data.duplicated().sum()
if duplicated_rows > 0:
    print("Số dòng lặp: ", duplicated_rows)
else:
    print("Không có dòng lặp")

Không có dòng lặp


Dữ liệu được thêm vào mỗi ngày nên tạm thời ta không nhận xét gì thêm ở đây

### Mỗi cột có ý nghĩa gì?

|Tên cột| Mô tả |
|:-:| :--|
|**Date**| Ngày lấy số liệu |
|**Country**| Nước |
|**Total Cases**| tổng số ca nhiễm mới từ trước đến nay |
|**New Cases**| số ca mới ngày hôm nay |
|**Total Deaths**| tổng số ca tử vong từ trước đến nay  |
|**New Deaths**| số ca tử vong mới ngày hôm nay|
|**Total Recovered**| tổng số ca phục hồi |
|**New Recovered**| số ca phục hồi trong ngày|
|**Active Cases**| số ca đang điều trị |
|**Serious Critical**| số ca nguy kịch |
|**Total Cases/1M pop**| số ca nhiễm / 1 triệu dân |
|**Deaths/1M pop**| số ca từ vong/ 1 triệu dân |
|**Total Tests**| tổng số xét nghiệm |
|**Tests/1M pop**| số xét nghiệm/1 triệu dân |
|**Population**| dân số |

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

Giờ ta xem qua kiểu dữ liệu của các cột nhằm phát hiện cột có kiểu dữ liệu chưa phù hợp

In [9]:
data.dtypes

Date                 object
Country              object
Total Cases           int64
New Cases           float64
Total Deaths        float64
New Deaths          float64
Total Recovered     float64
New Recovered       float64
Active Cases        float64
Serious Critical    float64
Tot Cases/1M pop    float64
Deaths/1M pop       float64
Total Tests         float64
Tests/1M pop        float64
Population          float64
dtype: object

Ta nhận thấy trừ cột `Data` và `Country` ra thì những cột còn lại nên chuyển về kiểu dữ liệu `int` (đúng về mặt ngữ nghĩa).

Bên cạnh đó, ta cần chuyển cột `Date` sang kiểu dữ liệu `datetime`

---

## Tiền Xử Lý Dữ Liệu

Bây giờ, trừ cột `Data` và `Country` ra thì những cột còn lại ta chuyển về kiểu dữ liệu `int`. Còn cột `Date` sẽ chuyển về kiểu dữ liệu `datetime`.

In [12]:
cat_cols = ['Country']
nume_cols = ['Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recovered',
                'New Recovered', 'Active Cases', 'Serious Critical', 'Tot Cases/1M pop',
                'Deaths/1M pop', 'Total Tests', 'Tests/1M pop', 'Population']

data['Date'] = datetime.datetime.strptime(data['Date'])

ValueError: Cannot convert non-finite values (NA or inf) to integer

### Sự phân bố giá trị của mỗi cột

#### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu số, ta tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

**Nhận xét**:

#### Với mỗi cột có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

Với các cột có kiểu dữ liệu dạng `categorical`, ta cần tính:

- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu
- Số lượng các giá trị khác nhau (không xét giá trị thiếu):
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu)

**Nhận xét**:

### Tiền xử lý dữ liệu

Ở Lab1, ta chỉ chọn 1 ngày để thực hiện yêu cầu của đề bày. Ở đây chúng ta chọn một ngày xác định **(19/4/2022)** để phân tích dữ liệu. 

Ta kiểm tra lại xem các cột nào đang còn thiếu dữ liệu